In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn import metrics as m
from thundersvm import SVC as svmgpu
import calculateWeightUsingGa2 as aresult
import pandas as pd
import itertools
import random

randomseed = 42
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [3]:
xtest = np.array(pd.read_csv("../dataset/xtest.txt"))
xtrain = np.array(pd.read_csv("../dataset/xtrain.txt"))
ytest_original = np.array(pd.read_csv("../dataset/ytest.txt")).ravel()
ytrain_original = np.array(pd.read_csv("../dataset/ytrain.txt")).ravel()

ytrain = ytrain_original.copy()
ytest = ytest_original.copy()


clf = []
acc = []
finalacc = []
ypredproba_all = []
ypredconfprob_all = []

rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain, ytrain)
print("original score", m.f1_score(ytest, rf.predict(xtest), average="weighted"))

original score 0.6863966818349385


<IPython.core.display.Javascript object>

In [4]:
# =================================================
# RF ===========================
# =================================================
rf = RandomForestClassifier(random_state=randomseed, n_estimators=10)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.f1_score(ytest, rfpred, average="weighted"))

clf.append(rf)
acc.append(m.f1_score(ytest, rfpred, average="weighted"))
ypredproba_all.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100)

0.6863966818349385


<IPython.core.display.Javascript object>

In [5]:
# =================================================
# svc ===========================
# =================================================

svc = svmgpu(random_state=randomseed, probability=True, C=100, gamma=0.0001)
svc.fit(xtrain, ytrain)

svcpred = svc.predict(xtest)
print(m.f1_score(ytest, svcpred, average="weighted"))

clf.append(svc)
acc.append(m.f1_score(ytest, svcpred, average="weighted"))
ypredproba_all.append(svc.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, svcpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100)

0.6852023910211884


<IPython.core.display.Javascript object>

In [6]:
# =================================================
# xgb ===========================
# =================================================

xgbc = xgb.XGBClassifier(random_state=randomseed, n_estimators=100)
xgbc.fit(xtrain, ytrain)

xgbpred = xgbc.predict(xtest)
print(m.f1_score(ytest, xgbpred, average="weighted"))


clf.append(xgbc)
acc.append(m.f1_score(ytest, xgbpred, average="weighted"))
ypredproba_all.append(xgbc.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, xgbpred)
confsumh = np.sum(confmat, axis=1)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
ypredconfprob_all.append(propconfmat / 100)

0.7335016228212894


<IPython.core.display.Javascript object>

In [7]:
# generate combinations of features 12,6
comb = list(itertools.combinations(np.arange(0, 12, 1), 7))

# generate 50 random numbers
randnums = []
for i in range(10):
    randnums.append(random.randrange(0, len(comb)))

print(randnums)

comb = np.array(comb)[randnums, :]


for i in range(len(comb)):
    print(i, " ==================== ", comb[i])

    rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
    rf.fit(xtrain[:, comb[i]], ytrain)
    rfpred = rf.predict(xtest[:, comb[i]])
    print(m.f1_score(ytest, rfpred,average='weighted'))

    clf.append(rf)
    acc.append(m.f1_score(ytest, rfpred,average='weighted'))
    ypredproba_all.append(rf.predict_proba(xtest[:, comb[i]]))

    confmat = m.confusion_matrix(ytest, rfpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100)

    xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
    xgbmodel.fit(xtrain, ytrain)
    xgbmodelpred = xgbmodel.predict(xtest)
    print(m.f1_score(ytest, xgbmodelpred,average='weighted'))

    clf.append(xgbmodel)
    acc.append(m.f1_score(ytest, xgbmodelpred,average='weighted'))
    ypredproba_all.append(xgbmodel.predict_proba(xtest))

    confmat = m.confusion_matrix(ytest, xgbmodelpred)
    confsumh = np.sum(confmat, axis=0)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100)


[306, 620, 552, 785, 261, 184, 529, 122, 309, 464]
0  ====================  [ 0  2  3  5  9 10 11]
0.6571913065491298
0.7351489555824932
1  ====================  [ 1  3  4  7  8 10 11]
0.6501636372412671
0.7351489555824932
2  ====================  [1 2 4 6 7 8 9]
0.6480557637235402
0.7351489555824932
3  ====================  [ 4  5  6  7  8  9 11]
0.3780963395414345
0.7351489555824932
4  ====================  [0 2 3 4 5 8 9]
0.6688360698977405
0.7351489555824932
5  ====================  [ 0  1  3  6  7  9 10]
0.6898691702007088
0.7351489555824932
6  ====================  [ 1  2  3  7  8 10 11]
0.6611299432792372
0.7351489555824932
7  ====================  [ 0  1  2  7  8  9 11]
0.706300817378268
0.7351489555824932
8  ====================  [ 0  2  3  6  7  8 11]
0.6857839273015596
0.7351489555824932
9  ====================  [1 2 3 4 5 6 9]
0.6272143586082548
0.7351489555824932


<IPython.core.display.Javascript object>

In [8]:
import calculateWeightUsingGa2 as aresult
weightvalga=aresult.getbestvalues(acc)

finalval=0
for i in range(len(acc)):
    finalval += weightvalga[i]*ypredproba_all[i]

print('f1_score',m.f1_score(ytest,np.argmax(finalval,axis=1),average='weighted'))
print('accuracy_score',m.accuracy_score(ytest,np.argmax(finalval,axis=1)))
    


0
1
2
3
4
11.616855021717939 [0.68472474 0.62209948 0.889393   0.45361289 0.9041462  0.35790937
 0.93188797 0.32905075 0.95030558 0.25995697 0.95393722 0.51927589
 0.96036004 0.77051348 0.97030615 0.47728554 0.97455038 0.86423199
 0.9832436  0.65224788 0.99046028 0.31951158 0.99998143]
f1_score 0.7365099952049899
accuracy_score 0.735


<IPython.core.display.Javascript object>